<a href="https://colab.research.google.com/github/oomiyakita-kbc/kbc-ytdl/blob/main/kbc_ytdl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U yt-dlp

from google.colab import drive
drive.mount('/content/drive')

from google.oauth2 import service_account
import googleapiclient.discovery

credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/KBC大宮北高校放送部/14.youtube-downloader/youtube-downloader-420717-fdc577ac7f2c.json')

service = googleapiclient.discovery.build('sheets', 'v4', credentials=credentials)

spreadsheet_id = '1k-l4X4NAUZPb5QQbytocC0qGIiY6cpOKIKoG3d707lQ'

In [ ]:
import yt_dlp

range_ = 'ダウンロード!A2:A'
result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_).execute()
values = result.get('values', [])

failed_downloads = []

for url in values:
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192'
        }],
        'prefer_ffmpeg': True,
        'continue_on_error': True,
        'outtmpl': '/content/drive/MyDrive/KBC大宮北高校放送部/14.youtube-downloader/01.downloaded_wav/%(title)s.%(ext)s',
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url[0]])
    except yt_dlp.DownloadError as e:
        print(f"ダウンロード失敗: {url[0]}: {e}")
        failed_downloads.append(url[0])

if failed_downloads:
    print("ダウンロード失敗一覧")
    for failed_title in failed_downloads:
        print(failed_title)
else:
    print("完了")
